In [8]:
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

from experiments.ModelLoader import load_vae_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def showandSave(img_tensor, title=None, i_cur=1):
    if img_tensor.dim() == 4:  # (B,C,H,W)
        img = make_grid(img_tensor, nrow=10, normalize=True, value_range=(0,1))
    else:  # (C,H,W)
        img = img_tensor.detach()

    # Convert (C,H,W) → (H,W,C)
    img = img.permute(1, 2, 0).cpu()

    # plt.imshow(img[:, :img.shape[1] // 2, :])
    plt.figure(figsize=(64, 64))
    plt.imshow(img)
    if title: plt.title(title)
    plt.axis("off")
    # plt.show()
    plt.savefig(f"G:\Python\VAE-latent-space-experiment\outputs\makegif\my_image{i_cur:04}.png", bbox_inches='tight', pad_inches=0)


✅ GIF saved as G:\Python\VAE-latent-space-experiment\outputs\makegif01.gif


In [ ]:
model = load_vae_model(checkpoint_path="./experiments/checkpoints/vae_checkpoint_epoch_16.pt", device=device, latent_dim=latent_dim)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
sample_mus = None
sample_var = None
for batch_idx, (x, _) in enumerate(test_loader, start=1):
    model.encoder.eval()
    model.eval()
    x = x.to(device, non_blocking=True)
    mu, logvar = model.encoder(x)
    sample_mus = mu
    sample_var = logvar
    print(mu.shape)
    # save_reconstructions(model=model, x=x, out_dir=out_dir, step = 16, device=device, variant="test")
    break
sample_mus = sample_mus.detach()
sample_var = sample_var.detach()

In [ ]:
import random
import torch

# Number of interpolation steps
num_steps = 7

for _ in range(30):
    # Pick two random indices
    idx_start = random.randint(1, 30)
    idx_end = random.randint(1, 30)

    # Unique identifier for this interpolation
    interpolation_id = idx_start * 100 + idx_end

    # Get start and end latent vectors
    latent_start = sample_mus[idx_start]
    latent_end = sample_mus[idx_end]

    # Initialize images with the starting latent
    images = [latent_start.unsqueeze(0)]
    print(latent_start.unsqueeze(0).shape)

    # Interpolation loop
    for step in range(num_steps + 1):
        alpha = step / num_steps  # fraction between start and end
        interpolated = (1 - alpha) * latent_start + alpha * latent_end
        print(interpolated.unsqueeze(0).shape)
        images.append(interpolated.unsqueeze(0))

    # Add the final latent
    images.append(latent_end.unsqueeze(0))

    # Create batch
    batch = torch.cat(images, dim=0)

    # Decode and save results
    showandSave(model.decoder(batch), i_cur=interpolation_id)
